In [1]:
!pip install tensorflow


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df=pd.read_csv('train.csv')
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
tweets=df['text']
y=df['target']

tweet pre-processing

In [5]:
one=tweets[6]
one

'#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas'

In [6]:
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

stops = set(stopwords.words('english'))
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
lemmatizer=WordNetLemmatizer()



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Somya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Somya\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
def preprocessing(tweet):
    tweet= re.sub(r'^RT[\s]+', '', tweet)
    tweet= re.sub(r'https?://[^\s\n\r]+', '', tweet)
    tweet= re.sub(r'#', '', tweet)
    
    tweet_tokens = tokenizer.tokenize(tweet)
    
    clean_tweet=[]
    for words in tweet_tokens:
        if(words not in string.punctuation and words not in stops):
            clean_tweet.append(words)
    lem_tweet=[]
    for words in clean_tweet:
        lem_tweet.append(lemmatizer.lemmatize(words))
    
    processed_tweet=' '.join(lem_tweet)
    
    return processed_tweet

    
    


In [8]:
X_text=[]
for tweet in tweets:
    X_text.append(preprocessing(tweet))

In [17]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vectorizer=TfidfVectorizer()
X=tf_idf_vectorizer.fit_transform(X_text)
print(type(X))
print(X.shape)
print(X.dtype)

<class 'scipy.sparse._csr.csr_matrix'>
(7613, 13490)
float64


In [10]:
# from sklearn.decomposition import TruncatedSVD
# svd = TruncatedSVD(n_components=500, n_iter=7, random_state=123)
# X_reduced= svd.fit_transform(X)


# print("Original matrix shape:", X.shape)
# print("Reduced matrix shape:", X_reduced.shape)


In [18]:
X=X.astype(np.int64)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.33, random_state=1442)

In [23]:
print(X_train.dtype)
print(type(X_train))
print(y_train.dtype)
print(type(y))

int64
<class 'scipy.sparse._csr.csr_matrix'>
int64
<class 'pandas.core.series.Series'>


In [24]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

X_train = X_train.toarray()

# Convert Pandas Series to NumPy array
y_train = y_train.values

model=Sequential([
    Dense(100,activation='relu'),
    Dense(50,activation='relu'),
    Dense(10,activation='relu'),
    Dense(2,activation='sigmoid'),
])

from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001))
model.fit(X_train,y_train,epochs=1000)


Epoch 1/1000
160/160 [==============================] - 2s 12ms/step - loss: 0.6893
Epoch 2/1000
160/160 [==============================] - 2s 12ms/step - loss: 0.6829
Epoch 3/1000
160/160 [==============================] - 2s 12ms/step - loss: 0.6825
Epoch 4/1000
160/160 [==============================] - 2s 12ms/step - loss: 0.6823
Epoch 5/1000
160/160 [==============================] - 2s 12ms/step - loss: 0.6823
Epoch 6/1000
160/160 [==============================] - 2s 12ms/step - loss: 0.6821
Epoch 7/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6819
Epoch 8/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6816
Epoch 9/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6814
Epoch 10/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6812
Epoch 11/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6809
Epoch 12/1000
160/160 [==============================] - 2s 13ms/step - lo

160/160 [==============================] - 2s 13ms/step - loss: 0.6759
Epoch 98/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6759
Epoch 99/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6759
Epoch 100/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6759
Epoch 101/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6759
Epoch 102/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6759
Epoch 103/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6761
Epoch 104/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6758
Epoch 105/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6759
Epoch 106/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 107/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6758
Epoch 108/1000
160/160 [==============================] - 2s 13ms/step 

160/160 [==============================] - 2s 12ms/step - loss: 0.6758
Epoch 193/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6758
Epoch 194/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6758
Epoch 195/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6757
Epoch 196/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6755
Epoch 197/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6760
Epoch 198/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6757
Epoch 199/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6756
Epoch 200/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6758
Epoch 201/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6757
Epoch 202/1000
160/160 [==============================] - 2s 11ms/step - loss: 0.6757
Epoch 203/1000
160/160 [==============================] - 2s 11ms/ste

160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 288/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6758
Epoch 289/1000
160/160 [==============================] - 2s 14ms/step - loss: 0.6757
Epoch 290/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 291/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 292/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6756
Epoch 293/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 294/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6758
Epoch 295/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 296/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6758
Epoch 297/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 298/1000
160/160 [==============================] - 2s 13ms/ste

160/160 [==============================] - 2s 14ms/step - loss: 0.6757
Epoch 383/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 384/1000
160/160 [==============================] - 2s 15ms/step - loss: 0.6757
Epoch 385/1000
160/160 [==============================] - 2s 15ms/step - loss: 0.6757
Epoch 386/1000
160/160 [==============================] - 2s 14ms/step - loss: 0.6756
Epoch 387/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 388/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 389/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 390/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 391/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 392/1000
160/160 [==============================] - 2s 13ms/step - loss: 0.6757
Epoch 393/1000
160/160 [==============================] - 2s 13ms/ste

KeyboardInterrupt: 

In [25]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

models = {}
models['LogisticRegression'] = LogisticRegression(C=5, max_iter=1000)
models['RidgeClassifier'] = RidgeClassifier(alpha=1.0, solver="sparse_cg")
models['KNeighborsClassifier'] = KNeighborsClassifier(n_neighbors=100)


for model_name, model in models.items():
    print(f"Fitting {model_name}...")
    model.fit(X_train, y_train)


    y_pred_cv = model.predict(X_cv)
  

    accuracy_cv = metrics.accuracy_score(y_cv, y_pred_cv)
    
    print(f"Training accuracy for validation set: {accuracy_cv}")
    
    print()

Fitting LogisticRegression...
Training accuracy for validation set: 0.565857540787903

Fitting RidgeClassifier...
Training accuracy for validation set: 0.565857540787903

Fitting KNeighborsClassifier...
Training accuracy for validation set: 0.5662554715479506



C:\Users\Somya\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
